In [14]:
from pinecone import Pinecone as PineconeClient
from pinecone.grpc import PineconeGRPC
from pinecone import ServerlessSpec
from dotenv import dotenv_values
# from langchain_community.chat_models import ChatOpenAI
from langchain_openai import ChatOpenAI
#from langchain_community.embeddings import CohereEmbeddings
from langchain_cohere import CohereEmbeddings
from langchain_community.vectorstores import Pinecone 
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
import requests
import json
import ray

connecto to pinecone api

In [8]:
config = dotenv_values(".env")
env_key = config["PINE_CONE_ENV_KEY"]
api_key = config["PINE_CONE_API_KEY"]
openai_api_key=config["OPENAI_API_KEY"]
cohere_api_key = config["COHERE_API_KEY"]
#pc_index = config["INDEX_NAME"]
print(f"openai key = {openai_api_key}")

openai key = sk-proj-3YRWRWnvcDDLVT9Ob3yQT3BlbkFJFBjjYn2yJfNoOZ1pFL0f


pinecone client and embeddings

In [9]:
pineCone = PineconeClient(
    api_key=api_key
)

start to load the data

In [4]:
url = "https://huggingface.co/api/datasets/Cohere/wikipedia-22-12-en-embeddings/parquet/default/train"
response = requests.get(url)
input_files = json.loads(response.content)
columns = ['id', 'title', 'text', 'url', 'emb']
ds = ray.data.read_parquet(input_files[1:100], columns=columns) 
print("Dataset:")
print(ds)
print("\n")

/home/jandogonzales/Development/anaconda3/envs/trailz/lib/python3.11/site-packages/ray/data/datasource/parquet_datasource.py:242: FutureWarning: Passing 'use_legacy_dataset' is deprecated as of pyarrow 15.0.0 and will be removed in a future version.
  pq_ds = pq.ParquetDataset(


Metadata Fetch Progress 0:   0%|          | 0/16 [00:00<?, ?it/s]

2024-04-18 12:43:00,187	INFO worker.py:1715 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 


Parquet Files Sample 0:   0%|          | 0/2 [00:00<?, ?it/s]

Dataset:
Dataset(
   num_blocks=792,
   num_rows=13761396,
   schema={
      id: int32,
      title: string,
      text: string,
      url: string,
      emb: list<item: float>
   }
)




(ReadParquet pid=264919) /home/jandogonzales/Development/anaconda3/envs/trailz/lib/python3.11/site-packages/ray/data/_internal/arrow_block.py:148: FutureWarning: promote has been superseded by promote_options='default'.
(ReadParquet pid=264919)   return transform_pyarrow.concat(tables)
(ReadParquet pid=265442) /home/jandogonzales/Development/anaconda3/envs/trailz/lib/python3.11/site-packages/ray/data/_internal/arrow_block.py:148: FutureWarning: promote has been superseded by promote_options='default'.
(ReadParquet pid=265442)   return transform_pyarrow.concat(tables)


In [5]:
ds.schema()
count = 0
for row in ds.iter_rows():
    print(row)
    count = count + 1
    if count > 0:
        break

2024-04-18 13:25:06,961	INFO set_read_parallelism.py:115 -- Using autodetected parallelism=744 for stage ReadParquet to satisfy output blocks of size at most DataContext.get_current().target_max_block_size=128.0MiB.
2024-04-18 13:25:06,962	INFO streaming_executor.py:112 -- Executing DAG InputDataBuffer[Input] -> TaskPoolMapOperator[ReadParquet]
2024-04-18 13:25:06,963	INFO streaming_executor.py:113 -- Execution config: ExecutionOptions(resource_limits=ExecutionResources(cpu=None, gpu=None, object_store_memory=None), exclude_resources=ExecutionResources(cpu=0, gpu=0, object_store_memory=0), locality_with_output=False, preserve_order=False, actor_locality_enabled=True, verbose_progress=False)
2024-04-18 13:25:06,964	INFO streaming_executor.py:115 -- Tip: For detailed progress reporting, run `ray.data.DataContext.get_current().execution_options.verbose_progress = True`


Running 0:   0%|          | 0/99 [00:00<?, ?it/s]

{'id': 139004, 'title': 'Papua New Guinea', 'text': 'There are several active volcanoes, and eruptions are frequent. Earthquakes are relatively common, sometimes accompanied by tsunamis. On 25 February 2018, an earthquake of magnitude 7.5 and depth of 35 kilometres struck the middle of Papua New Guinea. The worst of the damage was centred around the Southern Highlands region. Papua New Guinea is one of the few regions close to the equator that experience snowfall, which occurs in the most elevated parts of the mainland.', 'url': 'https://en.wikipedia.org/wiki?curid=22943', 'emb': [0.34855830669403076, -0.3931165635585785, -0.48193711042404175, 0.11413213610649109, -0.052384376525878906, -0.19725455343723297, 0.1848202496767044, -0.2045878916978836, 0.23143094778060913, 0.4025367200374603, -0.4508623480796814, 0.028451789170503616, -0.22486717998981476, -0.5817099213600159, -0.1718149185180664, 0.2288116067647934, 0.28102022409439087, 0.29903554916381836, -0.4376875162124634, -0.1948171

create the index for wikipedia data


<br>
indexes = pc.list_indexes().indexes<br>
print("Indexes:")<br>
print(indexes)<br>
print("\n")<br>


get the index from existing srcs


<br>
vectorstore = Pinecone.from_existing_index(index_name=pc_index,<br>
        embedding=embeddings)<br>
retriever = vectorstore.as_retriever()<br>
print("Vector store:")<br>
print(vectorstore)<br>
print("\n")<br>
def fetch_wiki_page(id):<br>
    url = f"https://en.wikipedia.org/w/api.php?action=query&prop=extracts&format=json&pageids={id}"<br>
    response = requests.get(url)<br>
    data = response.json()<br>
    page_content = list(data['query']['pages'].values())[0]['extract']<br>
    return page_content<br>
def fetch_url(x):<br>
    urls = [doc.metadata['url'] for doc in x['context']]<br>
    ids = [url.split('=')[-1] for url in urls]<br>
    contents = [fetch_wikipedia_page(id)[:32000] for id in ids]<br>
    return {"context": contents, "question": x["question"]}<br>


RAG prompt

template = 
Answer the question based only on the following context:<br>
    {context}<br>
    Question: {question}<br>
<br>
rompt = ChatPromptTemplate.from_template(template)

RAG model<br>
odel = ChatOpenAI(temperature=0, model="gpt-4-1106-preview", openai_api_key=openai_api_key)


<br>
chain = (<br>
    RunnableParallel({"context": retriever, "question": RunnablePassthrough()})<br>
    | RunnableLambda(fetch_url)<br>
    | prompt<br>
    | model<br>
    | StrOutputParser()<br>
)<br>


In [6]:
# upsert the data
from pinecone.grpc import PineconeGRPC
from pinecone import ServerlessSpec

pc = PineconeGRPC()
index_name = 'cohere-wikipedia'

# ensure index DNE
indexes = pc.list_indexes().indexes
names = [_['name'] for _ in indexes]
print("Names:")
print(names)
print("\n")

Names:
['trailz-ai', 'cohere-wikipedia']




In [7]:
# insert the new wiki index
if index_name not in names:
    pc.create_index(
        name=index_name,
        dimension=768,
        metric="cosine",
        spec=ServerlessSpec(cloud='aws', region='us-west-2'))


In [8]:
import numpy as np
from retry import retry
from tqdm.auto import tqdm

# process the data into vector format
def create_pc_dataset(dataset):
    pcDataList = []
    for row in dataset.iter_rows():
        newRow = {}
        newRow['_id'] = row['id']
        newRow['metadata'] = {}
        newRow['metadata']['title'] = row['title']
        newRow['metadata']['text'] = row['text']
        newRow['vector'] = row['emb']
        pcDataList.append(newRow)
    return pcDataList
        
def upload_batches(dataset):
    batch_size=350

# insert records async
def upload(batch):
    client = PineconeGRPC()
    index = client.Index(index_name)

    # sets the returned and error vectors
    total_vectors = 0
    num_failures = 0

    # data = process_data(large_batch).to_dict(orient='records')
    data = batch.to_dict(orient='records')

    # this will retry up to 2 times, exponential wait increase from min to 4-10
    @retry(stop=stop_after_attempt(2), wait=wait_exponential(multiplier=1, min=4, max=10))
    def send_batch(batch):
        return index.upsert(vectors=batch)

    try:
        result = send_batch(data)
        total_vectors += result.upserted_count
    except Exception as e:
        logging.exception(e)
        num_failures += len(data)
    return {'upserted': np.array([total_vectors]), 'errors': np.array([num_failures])}

class Upserter:
    def __call__(self, large_batch):
        return upload_batches(large_batch)

In [13]:
# set the env var for ray memory error
%env RAY_memory_monitor_refresh_ms=0

env: RAY_memory_monitor_refresh_ms=0


In [ ]:
# run the upload batches
from datetime import datetime
from datasets import Dataset
import pickle

%env RAY_memory_monitor_refresh_ms=0

# create the new dataset
pcDataList = create_pc_dataset(ds)
pcDataset = Dataset.from_list(pcDataList)

# create a pkl file for use later so we don't keep reloading
pkl_data = 'pkl_data'
with open(pkl_data+'/pc_dataset.pkl', 'wb') as f:
    pickle.dump(pcDataset, f)

# upload the new ds in batches

# new_ds = ds.map_batches(
#     Upserter,
#     batch_size=batch_size,
#     batch_format='pandas',
#     zero_copy_batch=True,
#     concurrency=1)

# before = datetime.now()
# summary = new_ds.materialize().sum(['upserted', 'errors'])

# summary
# duration = datetime.now() - before
# print({k: f"{v: ,}" for k,v in summary.items()})
# print(f"Duration = {duration}")

2024-04-18 12:25:48,383	INFO set_read_parallelism.py:115 -- Using autodetected parallelism=744 for stage ReadParquet to satisfy output blocks of size at most DataContext.get_current().target_max_block_size=128.0MiB.
2024-04-18 12:25:48,384	INFO streaming_executor.py:112 -- Executing DAG InputDataBuffer[Input] -> TaskPoolMapOperator[ReadParquet]
2024-04-18 12:25:48,385	INFO streaming_executor.py:113 -- Execution config: ExecutionOptions(resource_limits=ExecutionResources(cpu=None, gpu=None, object_store_memory=None), exclude_resources=ExecutionResources(cpu=0, gpu=0, object_store_memory=0), locality_with_output=False, preserve_order=False, actor_locality_enabled=True, verbose_progress=False)
2024-04-18 12:25:48,385	INFO streaming_executor.py:115 -- Tip: For detailed progress reporting, run `ray.data.DataContext.get_current().execution_options.verbose_progress = True`


env: RAY_memory_monitor_refresh_ms=0


Running 0:   0%|          | 0/99 [00:00<?, ?it/s]


<br>
embeddings = CohereEmbeddings(model="multilingual-22-12", cohere_api_key=cohere_api_key)<br>
print("Embeddings:")<br>
print(embeddings)<br>
print("\n")<br>


In [10]:
# create the cohere embeddings
from pinecone.grpc import PineconeGRPC
from pinecone import ServerlessSpec

pc = PineconeGRPC()
index_name = 'cohere-wikipedia'
print(f"Cohere = {cohere_api_key}")
embeddings = CohereEmbeddings(model="multilingual-22-12", cohere_api_key=cohere_api_key)
vectorstore = Pinecone.from_existing_index(index_name=index_name, embedding=embeddings)
retriever = vectorstore.as_retriever()

Cohere = y8rfGb9Ak9BSWnA6GcjDfoT4V0nXETZXzk5vf7TR


In [11]:
# fetch wiki using id
def fetch_wiki_page(id):
    url = f"https://en.wikipedia.org/w/api.php?action=query&prop=extracts&format=json&pageids={id}"
    response = requests.get(url)
    data = response.json()
    page_content = list(data['query']['pages'].values())[0]['extract']
    return page_content

def fetch_url(x):
    urls = [doc.metadata['url'] for doc in x['context']]
    ids = [url.split('=')[-1] for url in urls]
    contents = [fetch_wiki_page(id)[:32000] for id in ids]
    return {"context": contents, "question": x["question"]}

In [13]:
# RAG Prompt
from langchain_core.messages import HumanMessage, SystemMessage
from langchain.load.dump import dumps

template = """Answer the question based only on the following context:
{context}
Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

# RAG Model
#    | RunnableLambda(fetch_url)
model = ChatOpenAI(temperature=0, model="gpt-4-1106-preview", openai_api_key=openai_api_key)
chain = (
    RunnableParallel({"context": retriever, "question": RunnablePassthrough()})
    | prompt
    | model
    | StrOutputParser()
)

# Invoke a question to the chain
#    SystemMessage(content="You're a great assistant"),
# message = HumanMessage(content="What is film noir?")
# chain_result = chain.invoke("what is film noir?")
# chain_result = chain.invoke("what is film noir?")
messages = [
    SystemMessage(content="You are Micheal Jordan."),
    HumanMessage(content="Which shoe manufacturer are you associated with?"),
]
# chain_result = chain.invoke(messages)
chain_result = chain.invoke("What is film noir?")
# chain_result = chain.run("What is film noir?")
print("Chain result output:")
print(chain_result)

Chain result output:
Film noir is a category of film that has provoked ongoing debate regarding its definition and classification. It is characterized by a set of attributes that include being oneiric (dreamlike), strange, erotic, ambivalent, and cruel, as first described by French critics and Étienne Chaumeton in their 1955 book "Panorama du film noir américain 1941–1953." However, not every film noir necessarily includes all these attributes to the same degree.

Critics and scholars have variously identified film noir by its tragic or bleak conclusions, distinctive visual style, plot and character types, mood, and attitude. There is no definitive set of characteristics that can be universally applied to all films noir. The genre is also known for its complex characters and plotlines, crime and violence, mystery, and moral ambivalence.

Neo-noir is a related category that reflects many characteristics of classic film noir but also emphasizes a socio-critical perspective, often highlig